# Indiavix

In [2]:
import requests
import pandas as pd
from lxml import html
from io import StringIO
from datetime import datetime as dt, timedelta
from IPython.display import Markdown, display
from tabulate import tabulate

In [3]:
def get_vix(oprn_dt):
    today_str = (oprn_dt).strftime("%d-%b-%Y")
    oneyearago_str = (oprn_dt+timedelta(days=-364)).strftime("%d-%b-%Y")
    url = f"https://www.nseindia.com/products/dynaContent/equities/indices/hist_vix_data.jsp?&fromDate={oneyearago_str}&toDate={today_str}"
    res = requests.get(url)
    print("Operation status: ", url, res.status_code)
    tree = html.fromstring(res.text)
    csvdata = tree.xpath(r'//*[@id="csvContentDiv"]')[0].text
    df = pd.read_csv(StringIO(csvdata), lineterminator =":")
    df.columns = [i.strip() for i  in df.columns]
#     print(df.columns)
    df['Pct Rank']=df['% Change'].rank(pct=True)
    df['Close Rank']=df['Close'].rank(pct=True)
    
    pct_stats = {
        "pct_mean"                : df['% Change'].mean(),
        "pct_median"              : df['% Change'].median(), 
        "pct_std"                 : df['% Change'].std(),
        "pct_q10"                 :df['% Change'].quantile(0.1),
        "pct_q90"                 : df['% Change'].quantile(0.9),
        "pct_lastpercentilescore" : df['Pct Rank'].iloc[-1:].values[0]*100
    }
    
    close_stats = {
        "close_mean"                : df['Close'].mean(),
        "close_median"              : df['Close'].median(), 
        "close_std"                 : df['Close'].std(),
        "close_q10"                 : df['Close'].quantile(0.1),
        "close_q90"                 : df['Close'].quantile(0.9),
        "close_lastpercentilescore" : df['Close Rank'].iloc[-1:].values[0]*100
    }
    
    return df, pct_stats, close_stats

In [4]:
report_md_str=''
def add_to_report(string):
    global report_md_str
#     display(Markdown(string))
    report_md_str+=string+"  \n"
    
def prepare_report(df, pct_stats, close_stats):
    global report_md_str
    report_md_str = ''
    add_to_report('## Today:  ')
    add_to_report('---  ')
    add_to_report(tabulate(df.iloc[-1:], tablefmt="pipe", headers="keys")+"  \n  \n")
    add_to_report('---  ')
    add_to_report('## History of "Closing Volatility":  ')
    add_to_report('**Mean:** {0:.2f}, **Median:** {1:.2f}, **Standard Deviation:** {2:.2f}, **Quantile 10%:** {3:.2f}, **Quantile 90%:** {4:.2f}'.format(close_stats['close_mean'], close_stats['close_median'], close_stats['close_std'], close_stats['close_q10'], close_stats['close_q90']))
    add_to_report('**Percentile Rank of "Close Vol":** {:.2f}%'.format(close_stats['close_lastpercentilescore']))
    add_to_report('  \n---  ')
    add_to_report('## History of "% Change in Volatility":  ')
    add_to_report('**Mean:** {0:.2f}, **Median:** {1:.2f}, **Standard Deviation:** {2:.2f}, **Quantile 10%:** {3:.2f}, **Quantile 90%:** {4:.2f}'.format(pct_stats['pct_mean'], pct_stats['pct_median'], pct_stats['pct_std'], pct_stats['pct_q10'], pct_stats['pct_q90']))
    add_to_report('**Percentile Rank of "% Change":** {:.2f}%'.format(pct_stats['pct_lastpercentilescore']))
    return report_md_str


def save_report(report, date_str):
    with open(f"reports/{date_str}.md",'w') as f:
        f.write(report)


In [5]:
def get_vix_today():
    today = dt.now() # Not supported while market is open
    yesterday = today+timedelta(days=-1)
    df, pct_stats, close_stats = get_vix(yesterday)
    report = prepare_report(df, pct_stats, close_stats)
    save_report(report,  yesterday.strftime("%d-%b-%Y"))
    return df, pct_stats, close_stats, report, yesterday

In [6]:
df, pct_stats, close_stats, report, yesterday = get_vix_today()
display(Markdown(report))

Operation status:  https://www.nseindia.com/products/dynaContent/equities/indices/hist_vix_data.jsp?&fromDate=24-Sep-2018&toDate=23-Sep-2019 200


## Today:    
---    
|     | Date        |   Open |   High |   Low |   Close |   Prev. Close |   Change |   % Change |   Pct Rank |   Close Rank |
|----:|:------------|-------:|-------:|------:|--------:|--------------:|---------:|-----------:|-----------:|-------------:|
| 244 | 23-Sep-2019 |   15.4 |  17.64 | 14.21 | 16.7925 |          15.4 |     1.39 |       9.04 |   0.942857 |     0.538776 |  
  
  
---    
## History of "Closing Volatility":    
**Mean:** 17.36, **Median:** 16.63, **Standard Deviation:** 3.27, **Quantile 10%:** 14.12, **Quantile 90%:** 21.27  
**Percentile Rank of "Close Vol":** 53.88%  
  
---    
## History of "% Change in Volatility":    
**Mean:** 0.18, **Median:** -0.30, **Standard Deviation:** 5.28, **Quantile 10%:** -4.89, **Quantile 90%:** 6.85  
**Percentile Rank of "% Change":** 94.29%  


In [ ]:
def create 